In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from pandas import Series,DataFrame
# 获取远程数据集
import pandas_datareader  as web
from numpy.random import randn
# 默认情况下，matplotlib不支持中文显示，我们需要进行一下设置。 主题设置
# 设置字体,linux下用文泉驿字体windos下用SimHei字体(黑体),以支持中文显示
mpl.rcParams["font.family"] = ["WenQuanYi Micro Hei Mono"]
# 设置在中文字体时，能够正常的显示负号（-）。
mpl.rcParams["axes.unicode_minus"] = False
from sklearn.base import TransformerMixin
from nltk import word_tokenize
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import jieba # 因为是中文所以用结巴分词,
class NLTKBOW(TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return [{word:True for word in jieba.cut(document, cut_all=False)} for document in X]
df = pd.read_csv("./data/weibo/weibo_python_utf8.csv",encoding='utf-8',sep='`',index_col='Unnamed: 0')
tweets = df['text'].values
labels = df['labels'].values
labels[:5] = 0
pipeline = Pipeline([('bag-of-words',NLTKBOW()),
                     ('vectorizer',DictVectorizer()),
                     ('naive-bayes',BernoulliNB())])
scores = cross_val_score(pipeline,tweets,labels,scoring='f1')
print('score:{:.3f}'.format(np.mean(scores)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\10159\AppData\Local\Temp\jieba.cache
Loading model cost 0.608 seconds.
Prefix dict has been built succesfully.


score:0.986


In [ ]:
model = pipeline.fit(tweets,labels)
nb = model.named_steps['naive-bayes']
feature_probabilities = nb.feature_log_prob_ 
top_features = np.argsort(-feature_probabilities[1])[:50]
dv = model.named_steps['vectorizer']
for i,feature_index in enumerate(top_features):
    print(i,dv.feature_names_[feature_index],np.exp(feature_probabilities[1][feature_index]))

In [ ]:
from sklearn.externals import joblib
output_filename = os.path.join(os.path.expanduser('./'),'models','twitter','python_context.pkl')
# 接着,用joblib库的dump函数,与json库的dump函数功能类似.参数为模型本身(怕你忘了,捎带提下,模型名称为model)和输出文件名
joblib.dump(model,output_filename)